In [1]:
from pathlib import Path
import pandas as pd

base_path = Path(r"C:\Users\Jalynn\OneDrive\Documents\GitHub\Universal_EEG_Analyzer\StudyTwoGermaneLoadAnalysis\GammaResults2")

# Find all CSVs
low_files = list(base_path.rglob("*LowGermane*_avg_log_z_gamma_power_*.csv"))
high_files = list(base_path.rglob("*HighGermane*_avg_log_z_gamma_power_*.csv"))

print(f"Low: {len(low_files)}, High: {len(high_files)}")

def load_file(f, cond_label):
    pid = f.parent.name
    phase = "Train" if "train" in f.name.lower() else "Test"
    df = pd.read_csv(f)
    # rename columns with condition prefix
    df = df.rename(columns={
        "Fz": f"{cond_label}_Fz",
        "Cz": f"{cond_label}_Cz",
        "Pz": f"{cond_label}_Pz",
        "ACC_X": f"{cond_label}_Acc1",
        "ACC_Y": f"{cond_label}_Acc2",
        "ACC_Z": f"{cond_label}_Acc3",
    })
    df["PID"] = pid
    df["Phase"] = phase
    # only keep first row (each file has one row)
    return df.iloc[[0]]

# Load all low and high files
low_dfs = [load_file(f, "Cond1") for f in low_files]
high_dfs = [load_file(f, "Cond2") for f in high_files]

low_df = pd.concat(low_dfs, ignore_index=True)
high_df = pd.concat(high_dfs, ignore_index=True)

# Merge on PID + Phase
final_df = pd.merge(low_df, high_df, on=["PID", "Phase"], how="outer")

# Optional: rename Phase column to "Train/Test" for consistency
final_df = final_df.rename(columns={"Phase": "Train/Test"})

# Reorder columns
cols_order = ["PID", "Train/Test"] + \
    [c for c in final_df.columns if c not in ["PID", "Train/Test"]]
final_df = final_df[cols_order]

# Assuming `final_df` is your merged dataframe
# Drop the extra Condition columns if they exist
final_df = final_df.drop(columns=[col for col in final_df.columns if "Condition" in col], errors='ignore')

# Optionally reorder columns
cols_order = ["PID", "Train/Test"] + [c for c in final_df.columns if c not in ["PID","Train/Test"]]
final_df = final_df[cols_order]

# Save
final_df.to_csv(base_path / "Combined_LowHigh_Gamma.csv", index=False)
print("Saved combined CSV with", len(final_df), "rows")


Low: 66, High: 66
Saved combined CSV with 66 rows
